In [1]:
%pylab inline
from bigbrother.ministry import Ministry
from bigbrother.magnitudemetric import LuminosityFunction, TabulatedLuminosityFunction, AnalyticLuminosityFunction
from bigbrother.massmetric import MassFunction
from glob import glob
from collections import OrderedDict
import sys
import healpy as hp
import fitsio
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


/nfs/slac/g/ki/ki21/cosmo/jderose/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1318: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


## Compare the Buzzard luminosity function with other LFs

The first thing you usually want to do when using this code is to define a ministry object. Ministry objects contain information about cosmology, survey geometry, and how to optimize the I/O for running various calculations. They are the framework around which different types of catalogs are understood by the code.

In [ ]:
bzd = Ministry(0.3, 0.7, 0.7, 0.0, 2.0, 3*53.71)

The goal of this piece of code is to hide the differences between various types of catalogs using different catalog classes which inherit from the same BaseCatalog class. That way, whenever we want to do something with a galaxy catalog, or a halo catalog that action just needs to know about the BaseCatalog API.

In general a ministry object usually needs a few pieces of information:
###Metrics
Metrics are functions which calculate summary statistics on the catalogs generally corresponding to observables that we care about.

In [ ]:
bzdmetrics = [LuminosityFunction(bzd)]

We'll start simple and just look at a luminosity function in one redshift bin. The luminosity function metric defaults to using a redshift bin of $0.0\le z < 0.2$.

The second piece of information we need is the location of the relevant files on disk, as well as the types of those files. This information is provided by a dictionary called the filestruct. See below for an example.

In [ ]:
files = glob('/u/ki/jderose/desims/addgals/catalogs/Buzzard/Catalog_v1.1/truth/Buzzard_v1.1_truth.*.fit')
files[:5]

In [ ]:
g = fitsio.read(files[0])

In [ ]:
print(len(g))
print(g.dtype)

In [ ]:
bfs = {}
bfs['truth'] = files[:3]

The key of the filestruct specifies the type of file and the value is a list of all the files of that type that we want to use. Here I have designated a number of 'truth' files, but the key could have been anything as long as it is refered to consistently.

In order for the galaxy catalog object that we are about to create to know where to find various pieces of information that it needs in the files we just definied, we should define something called a fieldmap. This is a dictionary whose keys are the fields that are required by the metrics we want to use. These fields are called mapkeys in the code. 

The values of a fieldmap are also dictionaries. This time, the keys are the names of the fields corresponding to mapkeys in a catalog file, and the values are lists telling us in which types of files we can find the keys. In this case, the LuminosityFunction metric requires two mapkeys, luminosity and redshift. Thus we must define a fieldmap with these mapkeys as keys. The corresponding value of the luminosity key tells us that the luminosities are located in the 'AMAG' field of truth files. Likewise, redshifts are located in the 'Z' field of truth files.

In [ ]:
bzdmetrics[0].mapkeys

In [ ]:
fm = {'luminosity':OrderedDict([('AMAG',['truth'])]),
      'redshift':OrderedDict([('Z', ['truth'])])}

Now we have all of the information that our GalaxyCatalog object needs. This particular galaxy catalog that we are using now is a 'BCC' (Blind Cosmology Challege) catalog. Thus we set our simulations galaxy catalog using the function below.


In [ ]:
bzd.setGalaxyCatalog('BCC', bfs, fieldmap=fm)
bzd.galaxycatalog.metrics = bzdmetrics

In [ ]:
bzd.validate()

Now we can compare this to a the Bernardi 2013 (http://arxiv.org/abs/1304.7778) luminosity function as well as a double Schechter function plus a Gaussian tail at high luminosities.

In [ ]:
bf = np.array(['/u/ki/jderose/ki23/amatch/bernardi-test/anc/LF_SerExpAll.dat'])
bern = Ministry(0.286, 0.714, 0.7, 0.0, 2.0, 53.71)
bernLF = TabulatedLuminosityFunction(bern,fname=bf)
bernLF.loadLuminosityFunction()

In [ ]:
p0 = np.array([  1.50528627e-02,  -1.66000000e-01,   6.47466081e-03,  -1.52300000e+00,
  -2.00100000e+01,   2.0e-03,  -2.1500000e+01,   7.500000e-01])
dsgin = Ministry(0.286, 0.714, 0.7, 0.0, 2.0, 53.71)
dsgLF = AnalyticLuminosityFunction(dsgin)
dsgLF.calcNumberDensity(p0, form='doubleSchechterGaussian')

In [ ]:
bzdLF = bzd.galaxycatalog.metrics[0]

In [ ]:
bzdLF.visualize()

In [ ]:
f, ax = bzdLF.compare([bernLF, dsgLF], fylim=[-1.0, 1.0])
f.set_figheight(15)
f.set_figwidth(15)

## Plotting the Buzzard halo mass function

In this example we will plot the halo mass function of a BCC style halo catalog. Some halo files can be found as follows.

In [ ]:
hfiles = glob('/u/ki/jderose/desims/addgals/catalogs/Buzzard/Catalog_v1.1/halos/Buzzard_v1.1_halos_rotated.*.fit')
hfiles[:5]

Let's create a file structure for our halo catalog using these files

In [ ]:
hfs = {}
hfs['truth'] = hfiles[:3]

First of all, let's create a mass function object associated with the Buzzard ministry with default settings for mass and redshift bins.

In [ ]:
bzdmfcn = MassFunction(bzd)

Once we have this, we can find out what mapkeys the MassFunction metric requires

In [ ]:
bzdmfcn.mapkeys

We see that we need halo masses and redshifts. We now need to define a new field map for the halo catalog, similar to the one we defined for the galaxy catalog but with these mapkeys instead of luminosity and redshift. Let's look in the halo files to see where we can find mass and redshfit.

In [ ]:
halos = fitsio.read(hfiles[0])
print(halos.dtype)

There is a M200 field, and a Z field. These will work fine for our purposes.

In [ ]:
    hfm = {'mass':OrderedDict([('M200',['truth'])]),
          'redshift':OrderedDict([('Z', ['truth'])])}

In [ ]:
bzd.setHaloCatalog('BCC', hfs, fieldmap=hfm)
bzd.halocatalog.metrics = [bzdmfcn]
#We're resetting the ministry metrics here, because we don't want to recalculate the luminosity function
bzd.metrics = [bzdmfcn]

In [ ]:
bzd.validate()

This mass function will look strange below 3e13 $M_{\odot}$ because it measured on halos of three different resolution boxes.

In [ ]:
bzdmfcn.visualize()